# Profiling tools

With [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) we've already seen a way to measure the performance of parts of code. However, sometimes we want to zoom out and identify bottlenecks in a whole block of code.

Julia has a built-in [statistical profiler](https://goo.gl/Ycz4Td) in the standard library [`Profile`](https://docs.julialang.org/en/stable/stdlib/Profile/) (see also [here](https://docs.julialang.org/en/stable/manual/profile/)). 

In the following we want to use this profiler to identify the most time consuming parts of a function. We'll use ProfileView.jl and Juno to visualize the profiling results in a nice way.

## Profiling

Let's say we want to profile the following function in the file `myprogram.jl`:

```julia
function myfunction(n)
    for i = 1:n
        A = randn(100,100,20)
        m = maximum(A)
        Am = mapslices(sum, A; dims=2)
        B = A[:,:,5]
        Bsort = mapslices(sort, B; dims=1)
        b = rand(100)
        C = B.*b
    end
end
```

In [ ]:
include("myprogram.jl")

In [ ]:
myfunction(1)  # run once to trigger compilation

Profiling is as simple as prepending the function call by the `@profile` macro. (Don't forget to `using Profile` first.)

In [ ]:
using Profile
Profile.clear()  # in case we have any previous profiling data

@profile myfunction(10)

In principle there is `Profile.print()` to display the result. However it isn't particularily easy to parse.

In [ ]:
Profile.print()

## ProfileView.jl

We can get a much nicer visualization using [ProfileView.jl](https://github.com/timholy/ProfileView.jl).

(Note that ProfileView doesn't really work in jupyter lab but only jupyter notebooks and the REPL)

Make sure you have ProfileView installed.

```julia
using Pkg
Pkg.add("ProfileView")
```

In [ ]:
using ProfileView

ProfileView.view()

* The horizontal axis is basically "amount of time"
* The vertical axis represents nesting depth

From this visual representation, we can very quickly learn several things about this function:

* The most deeply-nested call corresponds to the `mapslices(sort, B; dims=1)` call.

* `mapslices(sum, A; dims=2)` is considerably more expensive than `mapslices(sort, B; dims=1)`. (This is because it has to process more data.)

Go to [ProfileView.jl's website](https://github.com/timholy/ProfileView.jl) to learn more about the details of this diagram.

## Profiling in the Juno IDE

Juno website: http://junolab.org/

Profiling section in the Juno documentation: http://docs.junolab.org/latest/man/juno_frontend.html#Profiler-1

*Let's open the file `myprogram.jl` in Juno...*

# [TimerOutputs.jl](https://github.com/KristofferC/TimerOutputs.jl)

In [ ]:
using TimerOutputs

In [ ]:
function myfunction(n, to)
    @timeit to "loop" for i = 1:n
        @timeit to "initialization" A = randn(100,100,20)
        m = maximum(A)
        @timeit to "mapslices 1" Am = mapslices(sum, A; dims=2)
        B = A[:,:,5]
        @timeit to "mapslices 2" Bsort = mapslices(sort, B; dims=1)
        b = rand(100)
        C = B.*b
    end
end

In [ ]:
to = TimerOutput()

myfunction(10, to)

to